In [24]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import calendar

In [25]:
os.getcwd()
os.chdir('c:\\Users\\tyler\\OneDrive\\Documents\\Python\\NFL')
os.getcwd()

'c:\\Users\\tyler\\OneDrive\\Documents\\Python\\NFL'

# Import 2021 Picks

In [26]:
df = pd.read_csv('backend\\data\\predictions\\2021.csv')
df['date'] = pd.to_datetime(df['date'])
df = df[df['date'] >= datetime(2021, 9, 1)]

## Betting Analysis 

### Per Month

In [27]:
df['month'] = df['date'].dt.month
df['month'] = df['month'].apply(lambda x: calendar.month_abbr[x])
df.groupby(
    ['month']
)['profit'].agg(['sum', 'count'])

,sum,count
month,,
Dec,7.739951,15
Feb,0.000000,0
Jan,-7.164523,15
Nov,21.182656,24
Oct,-5.530839,22
Sep,-2.449586,16


### Per Week

In [31]:
df['week'] = df['date'].dt.weekofyear

df.groupby(
    ['week']
)['profit'].agg(['sum', 'count'])

C:\Users\tyler\AppData\Local\Temp/ipykernel_36632/2687597309.py:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['week'] = df['date'].dt.weekofyear


,sum,count
week,,
1,-3.675000,8
2,-2.596154,4
3,0.000000,0
4,0.138889,1
6,0.000000,0
36,-1.777586,4
37,-1.248848,4
38,-0.936310,6
39,-2.736842,7
